# Création d'un dataset pour train le fine tuning

# Interface graphique


In [155]:
import re
import unicodedata

def nettoyer(chaine):
    """
    Nettoie une chaîne de caractères en supprimant les signes diacritiques,
    les espaces et les caractères non alphabétiques, puis extrait les mots.

    Args:
        chaine (str): La chaîne à traiter.

    Returns:
        list: Liste des mots nettoyés.
    """
    # Supprimer les signes diacritiques
    chaine_sans_diacritiques = ''.join(
        c for c in unicodedata.normalize('NFD', chaine) if unicodedata.category(c) != 'Mn'
    )

    # Garder uniquement les lettres (et remplacer les séparateurs par des espaces)
    chaine_sans_caracteres_speciaux = re.sub(r"[^a-zA-Z\s]", " ", chaine_sans_diacritiques)

    # Diviser en mots et supprimer les éléments vides
    liste = [mot for mot in chaine_sans_caracteres_speciaux.split() if mot]
    return liste

# Exemple d'utilisation
chaine = """['key word":', 'telephone noir samsung"}']"""
mots_nettoyes = nettoyer(chaine)
print(mots_nettoyes)


['key', 'word', 'telephone', 'noir', 'samsung']


FONCTION RECHERCHE


In [156]:
import pandas as pd
import re
from typing import List, Tuple, Optional

# Chargement des données
data1 = pd.read_csv("amazon.csv")

def m(keywords1):
    return [re.escape(keyword) for keyword in keywords1]

keywords1=['telephone', 'inteligent']
# Poids pour chaque critère
w1, w2, w3 = 15, 7, 5

def recherche_par_score(keywords1: List[str], 
                        price_range: Optional[Tuple[float, float]] = None, 
                        data=data1,
                        top_n: int = 5,
                        retailer_profile: int = 0,
                        user_profil: int = 0):
    """
    Recherche les produits les plus pertinents en calculant un score de correspondance.
    
    Le score est normalisé pour éviter qu'un critère domine les autres.

    :param data: DataFrame contenant les informations produits.
    :param keywords1: Liste de mots-clés à chercher.
    :param price_range: Tuple (min_price, max_price) pour filtrer par prix (facultatif).
    :param top_n: Nombre de produits à retourner (par défaut 5).
    :return: Liste de dictionnaires contenant les produits les plus pertinents avec leur ID et score normalisé.
    """

    keywords = m(keywords1)
    num_keywords = len(keywords)  # Nombre de mots-clés

    # Remplacer les valeurs nulles par des chaînes vides
    data['product_name'] = data['product_name'].fillna("")
    data['category'] = data['category'].fillna("")
    data['about_product'] = data['about_product'].fillna("")
    
    # Initialiser une colonne pour le score
    data['score'] = 0
    
    # Calculer le score basé sur les occurrences des mots-clés
    for keyword in keywords:
        keyword_lower = keyword.lower()
        
        # Normaliser le score pour le nom du produit
        def score_product_name(name):
            words = name.lower().split()
            total_score = 0
            d = 1
            for i, word in enumerate(words):
                if keyword_lower in word:
                    total_score += max(1, 10 - i)  # Points décroissants (10 pour le 1er mot, 9 pour le 2e, etc.)
                    d = max(1, i)
            return total_score / d  # Normalisation

        data['score'] += (w1 / num_keywords) * data['product_name'].apply(score_product_name)
        
        # Normaliser le score pour la catégorie
        def score_categorie(cat):
            if not cat:
                return 0
            elements = cat.split(',')
            total_score = 0
            for i, element in enumerate(reversed(elements)):
                if keyword_lower in element.lower():
                    total_score += (i + 1)  # Dernier élément = plus de points
            return total_score
        
        data['score'] += (w2 / num_keywords) * data['category'].apply(score_categorie)
        
        # Normaliser le score pour la description du produit
        data['score'] += (w3 / num_keywords) * data['about_product'].str.count(f"(?i){keyword}")
    
    # Filtrer par plage de prix si spécifiée
    if price_range:
        min_price, max_price = price_range
        data = data[(data['discounted_price'].str.replace('₹', '').str.replace(',', '').astype(float) >= min_price)
                    & (data['discounted_price'].str.replace('₹', '').str.replace(',', '').astype(float) <= max_price)]
    total_weights = w1 + w2 + w3
    data['score'] /= total_weights
    # Trier par score décroissant
    sorted_data = data.sort_values(by='score', ascending=False)
    
    # Retourner les top_n produits sous forme de liste de dictionnaires avec ID et score
    top_n_products = sorted_data.head(top_n)[['product_name', 'score', 'product_id']]  # On garde aussi 'product_id' et 'score'
    
    # Convertir les résultats en format JSON-serializable (dictionnaires avec ID et Score)
    result = top_n_products[['product_id', 'score']].to_dict(orient='records')  # Inclure 'product_id' et 'score'
    
    return result

print(recherche_par_score(keywords1))

[{'product_id': 'B07VVXJ2P5', 'score': 0.09259259259259259}, {'product_id': 'B0B1YVCJ2Y', 'score': 0.0}, {'product_id': 'B0BBLHTRM9', 'score': 0.0}, {'product_id': 'B09LMMFW3S', 'score': 0.0}, {'product_id': 'B082LZGK39', 'score': 0.0}]


FONCTION NEGOCIE

In [165]:
import random

df= pd.read_csv("amazon.csv")
df = df.drop_duplicates(subset='product_id', keep='first')
df.set_index('product_id', inplace=True)


retailer_personalities = {
    "aggressive": {"flexibility": 0.5, "termination_tendency": 0.8, "starting_price_factor": 1.2},
    "flexible": {"flexibility": 1.2, "termination_tendency": 0.3, "starting_price_factor": 1.0},
    "balanced": {"flexibility": 1.0, "termination_tendency": 0.5, "starting_price_factor": 1.1},
}

# Initialize negotiation settings
def initialize_negotiation(product_price, max_discount_percentage):

    retailer_personality = random.choice(list(retailer_personalities.keys()))
    traits = retailer_personalities[retailer_personality]

    retailer_price = product_price * traits['starting_price_factor']  # Retailer's initial price
    min_price = product_price * max_discount_percentage

    retailer_negotiation_score = traits['flexibility']  # Retailer's base flexibility
    termination_tendency = traits['termination_tendency']  # Retailer's tendency to terminate negotiation
    
    user_importance_score = random.uniform(0.5, 1.5)  # User value (adjusts retailer's flexibility)
    adjusted_rns = retailer_negotiation_score * user_importance_score

    return retailer_price, adjusted_rns, user_importance_score, min_price, termination_tendency

# Retailer response logic
def retailer_response(retailer_price, user_offer, rns, offer_history, min_price, termination_tendency):
    # Track user's offer history
    offer_history.append(user_offer)
    
    
    # Check if user is repeatedly lowballing
    low_offer_count = sum(1 for offer in offer_history if offer < min_price*random.uniform(0.8, 1.1))
    
    # Retailer's concession diminishes for persistent lowball offers
    penalty_factor = max(0.5, 1 - low_offer_count * 0.1)  # Reduced flexibility with repeated low offers
    
    
    # Determine if the retailer accepts the offer
    if user_offer >= retailer_price:
        return retailer_price, True, offer_history  # Agreement reached
    
    # Retailer calculates concession
    concession = random.uniform(0.01, 0.05) * rns * penalty_factor
    new_price = max(retailer_price - (retailer_price * concession), min_price, user_offer)
    
   # Determine if the retailer accepts the offer
    if termination_tendency > random.random() and user_offer < min_price*random.uniform(0.8, 1.1):
        print("Retailer: Your offers are too low. Negotiation terminated.")
        return retailer_price, True, offer_history
    
    return new_price, False, offer_history  # Negotiation continues

# Negotiation agent for chatbot
def negociation_agent(id_produit, user, proposed_price,negociation_state):
    if id_produit=="":
        return("De quel produit parle t-on ?",user,negociation_state)
    # Check if id_produit is in user.negociation.keys()
    if not negociation_state:
        # Initialize negotiation settings
        row_series = df.loc[id_produit]
        product_price=float(row_series["actual_price"][1:].replace(",",""))
        max_discount_percentage=float(row_series["discount_percentage"][:-1])/100
        retailer_price, rns, user_importance_score, min_price, termination_tendency = initialize_negotiation(product_price=product_price, max_discount_percentage=max_discount_percentage)
        negociation_state = True
        user["negociation"]={
            "retailer_price": retailer_price,
            "rns": rns,
            "user_importance_score": user_importance_score,
            "min_price": min_price,
            "termination_tendency": termination_tendency,
            "round_count": 0,
            "offer_history": []
            }
        

    user_price = float(proposed_price)
    
    retailer_price, agreement, offer_history = retailer_response(user["negociation"]['retailer_price'], user_price, user["negociation"]['rns'], user["negociation"]['offer_history'], user["negociation"]['min_price'], user["negociation"]['termination_tendency'])
    user["negociation"]['retailer_price'] = retailer_price
    user["negociation"]['offer_history'] = offer_history
    user["negociation"]['round_count'] += 1

    if agreement or user["negociation"]['round_count'] >= 10:
        user["negociation"]['round_count'] = 0
        negociation_state= False
        return "Retailer: Your offers are too low. Negotiation terminated",user,negociation_state
    return str(retailer_price),user,negociation_state







In [169]:
from flask import Flask, request, jsonify, render_template
from mistralai import Mistral
import functools
import ast
import json

df= pd.read_csv("amazon.csv")
df = df.drop_duplicates(subset='product_id', keep='first')
df.set_index('product_id', inplace=True)


# Accéder à la Série de la ligne pour un certain ID
"""id_to_find = "B0B3MWYCHQ"
row_series = df.loc[id_to_find]
print(row_series["product_name"])
"""
def recherche(d):
    keyword1 = nettoyer(d["key word"])
    print(keyword1)
    resultat=recherche_par_score(keyword1)
    print(resultat,type(resultat))
    return(resultat)

"""d={"key word": "telephone intelligent"}
recherche(d)
"""

Id_product="B0B3MWYCHQ"
def negocie(d):
    result=negociation_agent(d["user"]["negociation"]["Id_product"],d["user"],d["price_demande"],d["negociation_state"])
    price=d["price_demande"]
    return (result) 


names_to_functions = {
  'recherche': functools.partial(recherche),
  'negocie': functools.partial(negocie)
}
tools = [
    {
        "type": "function",
        "function": {
            "name": "recherche",
            "description": "look for the product searched and the price ",
            "parameters": {
                "type": "object",
                "properties": {
                    "key word": {
                        "type": "string",
                        "description": "2 key words describing the searched item",
                    },
                    "price": {
                        "type" :"string",
                        "description" : "maximal price asked by te client in dolllars"
                    }
                },
                "required": ["key word","price"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "negocie",
            "description": "if the client negociate, propose a new price decided by the retailer ",
            "parameters": {
                "type": "object",
                "properties": {
                    "price_demande": {
                        "type": "string",
                        "description": "price asked by the client",
                    }
                },
                "required": ["price asked"],
            },
        },
    }
]

app = Flask(__name__)
message = []

# Remplacez par votre propre clé API
api_key = 'Z74XbXre7TErlHeuJlIXYatlj2i9P054'
client = Mistral(api_key=api_key)

# ID de votre agent Mistral
agent_id = "ag:68495cb5:20241123:agent-1:e6ee1c3f"

user={"negociation":dict()}
user["negociation"]["Id_product"]=""
negociation_state=False


@app.route("/", methods=["GET"])
def index():    
    return render_template("index.html")
@app.route('/get_product_data', methods=['GET'])
def get_product_data():
    global negociation_state
    global user
    Id=user["negociation"]["Id_product"]
    # Accéder à la Série de la ligne pour un certain ID
    row_series = df.loc[Id]
    
    # Si le booléen negociation_state est vrai, retourner les données du produit
    if negociation_state:
        product_data = {
            "name": row_series["product_name"],
            "id": Id,
            "price": user["negociation"]["retailer_price"]
        }
        return jsonify(product_data), 200

    # Ajouter un gestionnaire pour d'autres cas si nécessaire
    return jsonify({"error": "Produit non disponible"}), 461


@app.route("/chat", methods=["POST"])

def chat():
    global user
    global negociation_state

    print(user,"2")
    prix="999"
    nom_du_produit="phone1"

    # Récupérer le message utilisateur envoyé via l'API
    user_message = request.json.get("message")
    if not user_message:
        return jsonify({"error": "Message is required"}), 400

    print(f"Message reçu : {user_message}")
    message.append({
                "role": "user",
                "content": user_message,
                })
    chat_response = client.agents.complete(
        agent_id=agent_id,
        messages = message,
        tools = tools,
        tool_choice="auto"
        )

    # Extraire la réponse de l'agent
    assistant_message = chat_response.choices[0].message
    content=assistant_message.content
    print("content ",content)
    message.append(assistant_message)
    result = ' '
    if assistant_message.tool_calls!=None:
        tool_call=assistant_message.tool_calls[0]
        function_name=tool_call.function.name
        print(function_name)
        args=json.loads(tool_call.function.arguments)
        print("args ",args)
        

        if function_name=="negocie":
            args["user"]=user
            args["negociation_state"]=negociation_state
            print(user)
        result=names_to_functions[function_name](args)

        print("resultat ",result)

        
        if function_name=="recherche":
            main_result=result[0]
            id_to_find,score = main_result["product_id"],main_result["score"] 
            print("ID:", id_to_find)
            row_series = df.loc[id_to_find]
            nom_du_produit=row_series["product_name"]
            prix=row_series["actual_price"]
            print("produit_name",nom_du_produit)
            print("prix", prix)
            user["negociation"]["Id_product"]=id_to_find
        else:
            prix,user,negociation_state=result
            print("next price",prix)
            #
        message.append({"role":"tool", "name":function_name, "content":nom_du_produit + str(prix), "tool_call_id":tool_call.id})
        
        response = client.agents.complete(
            agent_id=agent_id,
            messages = message
        )
        message.append(response.choices[0].message)
        content=response.choices[0].message.content
    return jsonify({"response": (content)})  
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Nov/2024 18:59:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Nov/2024 18:59:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Nov/2024 18:59:26] "GET /favicon.ico HTTP/1.1" 404 -


{'negociation': {'Id_product': ''}} 2
Message reçu : I want a fast charging samsung charger
content  
recherche
args  {'key word': 'fast charging samsung charger', 'price': '20'}
['fast', 'charging', 'samsung', 'charger']
[{'product_id': 'B0B2DJDCPX', 'score': 2.0833333333333335}, {'product_id': 'B0B2DJDCPX', 'score': 2.0833333333333335}, {'product_id': 'B08VFF6JQ8', 'score': 1.8703703703703705}, {'product_id': 'B08VF8V79P', 'score': 1.8240740740740742}, {'product_id': 'B096MSW6CT', 'score': 1.712962962962963}] <class 'list'>
resultat  [{'product_id': 'B0B2DJDCPX', 'score': 2.0833333333333335}, {'product_id': 'B0B2DJDCPX', 'score': 2.0833333333333335}, {'product_id': 'B08VFF6JQ8', 'score': 1.8703703703703705}, {'product_id': 'B08VF8V79P', 'score': 1.8240740740740742}, {'product_id': 'B096MSW6CT', 'score': 1.712962962962963}]
ID: B0B2DJDCPX
produit_name SWAPKART Fast Charging Cable and Data Sync USB Cable Compatible for iPhone 6/6S/7/7+/8/8+/10/11, 12, 13 Pro max iPad Air/Mini, iPod and

127.0.0.1 - - [24/Nov/2024 18:59:56] "POST /chat HTTP/1.1" 200 -


{'negociation': {'Id_product': 'B0B2DJDCPX'}} 2
Message reçu : Can I have It for 490 ?
content  
negocie
args  {'price_demande': '490'}
{'negociation': {'Id_product': 'B0B2DJDCPX'}}
resultat  ('490.71446901730457', {'negociation': {'retailer_price': 490.71446901730457, 'rns': 1.0203993685381025, 'user_importance_score': 0.8503328071150855, 'min_price': 289.41999999999996, 'termination_tendency': 0.3, 'round_count': 1, 'offer_history': [490.0]}}, True)
next price 490.71446901730457


127.0.0.1 - - [24/Nov/2024 19:00:13] "POST /chat HTTP/1.1" 200 -
127.0.0.1 - - [24/Nov/2024 19:00:18] "GET /get_product_data HTTP/1.1" 500 -
Traceback (most recent call last):
  File "c:\Users\noeca\Documents\Hackathon\Hockathon\hackaton\venv\Lib\site-packages\flask\app.py", line 1536, in __call__
    return self.wsgi_app(environ, start_response)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\noeca\Documents\Hackathon\Hockathon\hackaton\venv\Lib\site-packages\flask\app.py", line 1514, in wsgi_app
    response = self.handle_exception(e)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\noeca\Documents\Hackathon\Hockathon\hackaton\venv\Lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\noeca\Documents\Hackathon\Hockathon\hackaton\venv\Lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^

{'negociation': {'retailer_price': 490.71446901730457, 'rns': 1.0203993685381025, 'user_importance_score': 0.8503328071150855, 'min_price': 289.41999999999996, 'termination_tendency': 0.3, 'round_count': 1, 'offer_history': [490.0]}} 2
Message reçu : I want it for 470


127.0.0.1 - - [24/Nov/2024 19:00:39] "POST /chat HTTP/1.1" 500 -


content  
negocie
args  {'price_demande': '470'}
{'negociation': {'retailer_price': 490.71446901730457, 'rns': 1.0203993685381025, 'user_importance_score': 0.8503328071150855, 'min_price': 289.41999999999996, 'termination_tendency': 0.3, 'round_count': 1, 'offer_history': [490.0]}}


Traceback (most recent call last):
  File "c:\Users\noeca\Documents\Hackathon\Hockathon\hackaton\venv\Lib\site-packages\flask\app.py", line 1536, in __call__
    return self.wsgi_app(environ, start_response)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\noeca\Documents\Hackathon\Hockathon\hackaton\venv\Lib\site-packages\flask\app.py", line 1514, in wsgi_app
    response = self.handle_exception(e)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\noeca\Documents\Hackathon\Hockathon\hackaton\venv\Lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\noeca\Documents\Hackathon\Hockathon\hackaton\venv\Lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\noeca\Documents\Hackathon\Hockathon\hackaton\venv\Lib\site-packages\flask\app.py", line 917, in fu

# Reformatage et Tools


In [22]:
import re

def format_message(assistant_message):
    """
    Formate un message brut en HTML lisible :
    - Met en gras les parties entourées par **.
    - Ajoute des sauts de ligne pour les énumérations et sous-détails.

    Args:
        assistant_message (str): Message brut de l'assistant.

    Returns:
        str: Message formaté en HTML.
    """
    formatted_message = ""  # Message formaté
    
    # Découper le message en lignes
    lines = assistant_message.split("\n")
    
    for line in lines:
        line = line.strip()
        if not line:  # Ignorer les lignes vides
            continue

        # Si la ligne commence par un numéro suivi d'un point, c'est une énumération principale
        if re.match(r"^\d+\.", line):
            formatted_message += f"<p><strong>{line}</strong></p>"

        # Si la ligne commence par "-", c'est un sous-détail
        elif line.startswith("-"):
            formatted_message += f"<p>&nbsp;&nbsp;&bull; {line[1:].strip()}</p>"

        # Sinon, c'est un texte simple ou un paragraphe
        else:
            # Remplacer **texte** par <strong>texte</strong>
            line = re.sub(r"\*\*(.*?)\*\*", r"<strong>\1</strong>", line)
            formatted_message += f"<p>{line}</p>"

    return formatted_message
